# HOMEWORK 7

7.1 To Create a crawler using the webcrawler provided, have it crawl the first 200 pages from a base URL of your choosing


The below are the steps to create the crawler, that I followed from my lecture https://github.com/pschragger/big-data-python-class/blob/master/Lectures/Lecture_7_-_Link_Analysis.ipynb.

In [137]:
import scrapy 
import os
os.chdir("C:\\Users\\Pavani\\Desktop\\bigdata")

In [138]:
mkdir code

A subdirectory or file code already exists.


In [139]:
cd code

C:\Users\Pavani\Desktop\bigdata\code


In [140]:
%%cmd
scrapy startproject tutorial

Microsoft Windows [Version 10.0.15063]
(c) 2017 Microsoft Corporation. All rights reserved.

C:\Users\Pavani\Desktop\bigdata\code>scrapy startproject tutorial
Error: scrapy.cfg already exists in C:\Users\Pavani\Desktop\bigdata\code\tutorial

C:\Users\Pavani\Desktop\bigdata\code>

In [141]:
cd tutorial

C:\Users\Pavani\Desktop\bigdata\code\tutorial


In [142]:
%%cmd
scrapy genspider weather weather.com

Microsoft Windows [Version 10.0.15063]
(c) 2017 Microsoft Corporation. All rights reserved.

C:\Users\Pavani\Desktop\bigdata\code\tutorial>scrapy genspider weather weather.com
Spider 'weather' already exists in module:
  tutorial.spiders.weather

C:\Users\Pavani\Desktop\bigdata\code\tutorial>

In [143]:
cd tutorial/

C:\Users\Pavani\Desktop\bigdata\code\tutorial\tutorial


In [144]:
cd spiders/

C:\Users\Pavani\Desktop\bigdata\code\tutorial\tutorial\spiders


In [145]:
%%writefile weather.py
import scrapy

class WeatherItem(scrapy.Item):
    title = scrapy.Field()
    link = scrapy.Field()
    resp = scrapy.Field()
    
class WeatherSpider(scrapy.Spider):
    name = 'weather'
    allowed_domains = ['weather.com']
    start_urls = ['https://weather.com/']

    def parse(self, response):
        hxs = scrapy.Selector(response)
        titles = hxs.xpath('//ul/li')
        item = []
        for title in titles:
            obj = WeatherItem()
            obj["title"] = title.xpath("a/text()").extract()
            obj["link"] = title.xpath("a/@href").extract()
            obj["resp"] = response
            if obj["title"] != []:
                item.append(obj)

        return item

Overwriting weather.py


The following command saves the crawled urls in weather_txt.csv file

In [146]:
%%cmd
scrapy crawl weather -o weather_txt.csv -t csv

Microsoft Windows [Version 10.0.15063]
(c) 2017 Microsoft Corporation. All rights reserved.

C:\Users\Pavani\Desktop\bigdata\code\tutorial\tutorial\spiders>scrapy crawl weather -o weather_txt.csv -t csv

C:\Users\Pavani\Desktop\bigdata\code\tutorial\tutorial\spiders>

2017-11-19 01:57:26 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: tutorial)
2017-11-19 01:57:26 [scrapy.utils.log] INFO: Overridden settings: {'NEWSPIDER_MODULE': 'tutorial.spiders', 'FEED_URI': 'weather_txt.csv', 'SPIDER_MODULES': ['tutorial.spiders'], 'BOT_NAME': 'tutorial', 'ROBOTSTXT_OBEY': True, 'FEED_FORMAT': 'csv'}
2017-11-19 01:57:26 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2017-11-19 01:57:26 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware',
 'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloaderm

7.2 To create a Stochastic matrix from its resulting crawling as per https://cs7083.wordpress.com/2013/01/31/demystifying-the-pagerank-and-hits-algorithms/

In [154]:
import pandas as pd
import numpy as np

lst = []
raw = pd.read_csv("weather_txt.csv")
raw['link'] = 'http://www.weather.com'+raw['link']
r = raw['resp'][0]
lst.append((r.split()[-1]).split('>')[-2])
for i in range(len(raw)):
    if raw['resp'][i] == r:
        lst.append(raw['link'][i])
    else:
        r = raw['resp'][i]
        lst.append((r.split()[-1]).split('>')[-2])
        
item = list(pd.DataFrame(lst)[0].unique())

link = []
length = len(lst)
for i, val in enumerate(lst):
    if i < length-1:
        link.append((lst[i], lst[i+1]))

n = pd.DataFrame(index=item, columns=item)
m = n.replace(np.NaN, 0)
for i in link:
    m.loc[i] = 1.0

ar = np.array(m)
v = ar.sum(axis=1)

result = ar/v[:, np.newaxis]
matrix = np.nan_to_num(result)

matrix

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

7.3 Pass it through the Page Rank algorithm and provide the list of the top 5 page URLs in our sample

In [153]:
from numpy import *

def pagerank(H):
    n = len(H)
    w = zeros(n)
    rho = 1./n * ones(n);
    for i in range(n):
        if multiply.reduce(H[i]== zeros(n)):
            w[i] = 1
    newH = H + outer((1./n * w),ones(n))
 
    theta=0.85
    G = (theta * newH) + ((1-theta) * outer(1./n * ones(n), ones(n)))
    #print rho
    for j in range(10):
        rho = dot(rho,G)
        #print rho
    
    return list(rho)

In [156]:
#please create matrix again after interpreting pagerank algorithm
r = pagerank(matrix)
value = ((pd.DataFrame(r)).sort_values(0, ascending=False)).head(5)
index = list(value.index)
index

[35, 36, 37, 38, 5]

In [157]:
url = pd.DataFrame(item)
print "Top five URLs:"
for i in sort_index:
    print url[0][i]

Top five URLs:
http://www.weather.com/tv
http://www.weather.com/tv/personalities
http://www.weather.com//feedback.weather.com/customer/en/portal/articles
http://www.weather.comhttp://ibm.biz/BdH3av
http://www.weather.com/life/manage-notifications


7.4 Using the hits algorithm ( with a connectivity matrix )

In [158]:
def hits(A):
    n= len(A)
    Au= dot(transpose(A),A)
    Hu = dot(A,transpose(A))
    a = ones(n); h = ones(n)
    #print a,h
    for j in range(5):
        a = dot(a,Au)
        a= a/sum(a)
        h = dot(h,Hu)
        h = h/ sum(h)
    return h

In [159]:
value = pd.DataFrame(hits(ar))
sort = (value.sort_values(0, ascending=False)).head(5)
hit = list(sort.index)

In [160]:
import pandas as pd
url = pd.DataFrame(item)
print "Top 5 page URLs:"
for i in hit:
    print 'http://www.weather.com'+(url[0][i]).split('.com')[-1]

Top 5 page URLs:
http://www.weather.com/life/manage-notifications
http://www.weather.com/news/trending/video/divers-free-huge-sharks-caught-in-net
http://www.weather.com/life/local-advertise-self-serve
http://www.weather.com/storms/report-recover
http://www.weather.com/tv/the-weather-channel-live/video/watch-the-weather-channel-live?pl=pl-live-tv-now


7.5 Another ranking alogrithm

Tried to implement support vector machine for ranking but could not get it implemented on our webpage datasets. This is because I could not find training dataset to test my weather dataset.

http://scikit-learn.org/stable/modules/svm.html                                         
https://medium.com/machine-learning-101/chapter-2-svm-support-vector-machine-theory-f0812effc72


# References

https://en.wikipedia.org/wiki/PageRank                                
https://www.slideshare.net/Ankit007_/ranking-algorithms                         
http://www.totallycommunications.com/latest/search-engine-basics-crawling-indexing-ranking/                       
https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/                     
https://cs7083.wordpress.com/2013/01/31/demystifying-the-pagerank-and-hits-algorithms/                                
https://weather.com/                                                                                         
https://doc.scrapy.org/en/latest/intro/tutorial.html                                                                  
https://doc.scrapy.org/en/latest/intro/install.html#intro-install                                       